# Detection Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

from project_paths import paths 
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt 

from lane_detection_medium.inference import DetectionInference
from lane_detection_medium.evaluation import DetectionEvaluator
from lane_detection_medium.utils.fs import get_date_string, read_image, read_yolo_labels
from lane_detection_medium.utils.viz import render_bbox
from lane_detection_medium.types.detection_types import ImageDetections

%matplotlib inline

## Trained Model Loading

In [ ]:
EXP_NAME = "train-2023-07-034"

CHECKPOINT_DPATH = paths.yolo_dpath / "LaneMarkingsDetection" / EXP_NAME / "weights"
MODEL_FPATH = CHECKPOINT_DPATH / "best.pt"

inference = DetectionInference.from_file(
    str(MODEL_FPATH), 
    device="cuda:0", 
    img_size=(640, 640)
)

## Evaluation

In [ ]:
data_dpath = paths.yolo_dpath / "data" / "2023_07_03" / "test"
evaluator = DetectionEvaluator(model=inference, verbose=True)

In [ ]:
det_report = evaluator.evaluate(data_dpath)

det_report

## Evaluation Result Caching

In [ ]:
cache_dpath = paths.data / "detection_results" / f"{EXP_NAME}_{get_date_string()}"
cache_dpath.mkdir(parents=True, exist_ok=True)

det_report.to_csv(cache_dpath / "metric_report.cvs")

## Visualization

In [ ]:
pred_color_map = { 
    "solid_white": (255, 0, 0), 
    "break_white": (0, 0, 255), 
    "zebra": (255, 255, 0)
}

gt_color_map = { 
    "solid_white": (0, 255, 0), 
    "break_white": (255, 100, 100), 
    "zebra": (255, 255, 0)
}

img_fpaths = sorted(list((data_dpath / "images").glob("*.PNG")))
txt_fpaths = sorted(list((data_dpath / "labels").glob("*.txt")))

@interact 
def show_inference(index=IntSlider(val=0, min=0, max=len(img_fpaths) - 1)):
    test_image = read_image(img_fpaths[index]) 
    gt_np = read_yolo_labels(txt_fpaths[index])
    gt_dets = ImageDetections.from_yolo_labels(gt_np, *test_image.shape[:2])

    detection_result = inference.detect([test_image], conf=0.25)[0]

    canva = test_image.copy()

    plt.figure(figsize=(12, 12))

     # --- Ground Truth Rendering --- #
    for gt_det in gt_dets:
        label_name = inference.names_map[gt_det.label_id]
        render_bbox(
            canva, gt_det.bbox, label=label_name, color=gt_color_map[label_name], font_color=(255, 255, 255)
        )

    # --- Predicted results Rendering --- # 
    for det in detection_result:
        label_name = f"{det.label_name}: {det.conf:.2f}"
        render_bbox(
            canva, det.bbox, label=label_name, color=pred_color_map[det.label_name], font_color=(255, 255, 255)
        )

    plt.title(img_fpaths[index].name)
    plt.imshow(canva)
    plt.show()